In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import lightkurve as lk

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
f = "exoplanets/tess-s0010-1-1_173.957400_-29.156000_100x100_astrocut.fits"

In [ ]:
exo = tess_cpm.Source(f)

In [ ]:
exo.plot_cutout()

In [ ]:
exo.set_aperture(rowrange=[49, 52], colrange=[49, 52])
exo.plot_cutout(rowrange=[45, 55], colrange=[45, 55], show_aperture=True)

In [ ]:
exo.plot_pix_by_pix()

In [ ]:
exo.add_cpm_model()
exo.set_regs([0.2])
exo.holdout_fit_predict(k=20);

In [ ]:
# exo.plot_pix_by_pix(split=True, data_type="raw")
# exo.plot_pix_by_pix(split=True, data_type="prediction")
# exo.plot_pix_by_pix(split=True, data_type="cpm_prediction")
# exo.plot_pix_by_pix(split=True, data_type="poly_model_prediction")
exo.plot_pix_by_pix(split=True, data_type="cpm_subtracted_flux")
lc = exo.get_aperture_lc("cpm_subtracted_flux")

In [ ]:
level = 1.
splits = exo.get_aperture_lc("cpm_subtracted_flux", split=True)
clipped_above = np.concatenate([flux > (np.mean(flux) + level*np.std(flux)) for flux in splits])
clipped_below = np.concatenate([flux < (np.mean(flux) - level*np.std(flux)) for flux in splits])
outliers = clipped_above + clipped_below
outliers

In [ ]:
# lc = exo.get_aperture_lc("cpm_subtracted_flux")
plt.plot(exo.time, lc)
# sigma = np.std(lc)
# outliers = np.abs(lc) > 1.*sigma
plt.plot(exo.time[outliers], lc[outliers], "x", c="red")

In [ ]:
exo.holdout_fit_predict(k=20, mask=~outliers);
exo.plot_pix_by_pix("cpm_subtracted_flux")

In [ ]:
masked_lc = exo.get_aperture_lc("cpm_subtracted_flux")
plt.plot(exo.time, lc)
plt.plot(exo.time, masked_lc, alpha=0.3)

In [ ]:
apt_cpm = exo.get_aperture_lc(data_type="cpm_prediction")
apt_lc = exo.get_aperture_lc(data_type="cpm_subtracted_lc")
# exo.get_aperture_lc(split=False, data_type="cpm_subtracted_lc")

In [ ]:
# plt.plot(exo.time, apt_cpm, ".")
plt.plot(exo.time, apt_lc, ".")
# outliers = np.abs(apt_lc) > 1.5*np.std(apt_lc)
# print(np.sum(outliers))
# plt.plot(exo.time[outliers], apt_lc[outliers], "x", c="k")
# plt.plot(exo.time[~outliers], apt_lc[~outliers], "x", c="k")

# apt_lc[pre_clip] = np.nan
# clip = np.abs(apt_lc) > 1*np.std(apt_lc[pre_clip])
# plt.plot(exo.time[clip], apt_lc[clip], "x", c="r")
# np.any(clip)

In [ ]:
# exo.holdout_fit_predict(k=20, mask=~outliers);

In [ ]:
clipped_apt_lc = exo.get_aperture_lc(data_type="cpm_subtracted_lc")
plt.plot(exo.time, clipped_apt_lc, ".", label='Clipped Prediction')
plt.plot(exo.time, apt_lc, "-", label='no clipping')
plt.legend()

In [ ]:
cpm_regs = 10.0 ** np.arange(-10, 10)

In [ ]:
min_cpm_reg, cdpps = exo.calc_min_cpm_reg(cpm_regs, k=10)
plt.savefig("Exoplanet_CDPPS.png", dpi=200)

In [ ]:
cpm_regs

In [ ]:
print(min_cpm_reg)
exo.set_regs([min_cpm_reg])
exo.holdout_fit_predict(k=10);
flux_splits = exo.get_aperture_lc(data_type="raw", split=True)
splits = exo.get_aperture_lc(data_type="cpm_subtracted_lc", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(18, 8))
for time, split, f_split in zip(exo.split_times, splits, flux_splits):
    axs[0].plot(time, f_split, ".")
    axs[0].set_ylabel("Original Flux", fontsize=15)
    axs[1].plot(time, split)
    axs[1].set_ylabel("Detrended Flux", fontsize=15)
    axs[1].set_xlabel( "Time [BTJD]", fontsize=15)
plt.savefig("Exoplanet_LC.png", dpi=200)

In [ ]:
print(min_cpm_reg)
exo.set_regs([min_cpm_reg])
exo.holdout_fit_predict(k=10);
flux_splits = exo.get_aperture_lc(data_type="raw", split=True)
splits = exo.get_aperture_lc(data_type="cpm_subtracted_lc", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(18, 8))
for time, split, f_split in zip(exo.split_times, splits, flux_splits):
    axs[0].plot(time, f_split, ".")
    axs[0].set_ylabel("Original Flux", fontsize=15)
    axs[1].plot(time, split)
    axs[1].set_ylabel("Detrended Flux", fontsize=15)
    axs[1].set_xlabel( "Time [BTJD]", fontsize=15)
# plt.savefig("Exoplanet_LC.png", dpi=200)

In [ ]:
lc = lk.TessLightCurve(time=exo.time, flux=exo.get_aperture_lc(data_type="cpm_subtracted_lc")+1)
# lc.plot(normalize=False)
# lc.normalize()
# np.nanmedian(lc.flux)
# np.nanstd(lc.flux)
# np.abs(np.nanmedian(lc.flux)) < 0.5*np.nanstd(lc.flux)
plt.plot(lc.time, -1*lc.flatten().flux)
# a = lc.flatten()
# a

In [ ]:
split_cpm_subtracted_lc = exo.get_aperture_lc(split=True, data_type="cpm_subtracted_lc")
cpm_subtracted_lc = exo.get_aperture_lc(data_type="cpm_subtracted_lc")

for t, l in zip(exo.split_times, split_cpm_subtracted_lc):
    plt.plot(t, l)